# WordRank wrapper tutorial on Lee Corpus

WordRank is a new word embedding algorithm which captures the semantic similarities in a text data well. See this [notebook](Wordrank_comparisons.ipynb) for it's comparisons to other popular embedding models. This tutorial will serve as a guide to use the WordRank wrapper in gensim. You need to install [WordRank](https://bitbucket.org/shihaoji/wordrank) before proceeding with this tutorial.


# Train model

We'll use [Lee corpus](https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/test/test_data/lee.cor) for training which is already available in gensim. Now for Wordrank, two parameters `dump_period` and `iter` needs to be in sync as it dumps the embedding file with the start of next iteration. For example, if you want results after 10 iterations, you need to use `iter=11` and `dump_period` can be anything that gives mod 0 with resulting iteration, in this case 2 or 5.


In [1]:
from gensim.models.wrappers import Wordrank

wr_path = 'wordrank' # path to Wordrank directory
out_dir = 'model' # name of output directory to save data to
data = '../../gensim/test/test_data/lee.cor' # sample corpus

model = Wordrank.train(wr_path, data, out_dir, iter=11, dump_period=5)

FileNotFoundError: [Errno 2] No such file or directory: 'wordrank/glove/vocab_count': 'wordrank/glove/vocab_count'

Now, you can use any of the Keyed Vector function in gensim, on this model for further tasks. For example,

In [ ]:
model.most_similar('President')

In [ ]:
model.similarity('President', 'military')

As Wordrank provides two sets of embeddings, the word and context embedding, you can obtain their addition by setting ensemble parameter to 1 in the train method.

# Save and Load models
In case, you have trained the model yourself using demo scripts in Wordrank, you can then simply load the embedding files in gensim. 

Also, Wordrank doesn't return the embeddings sorted according to the word frequency in corpus, so you can use the sorted_vocab parameter in the load method. But for that, you need to provide the vocabulary file generated in the 'matrix.toy' directory(if you used default names in demo) where all the metadata is stored.

In [ ]:
wr_word_embedding = 'wordrank.words'
vocab_file = 'vocab.txt'

model = Wordrank.load_wordrank_model(wr_word_embedding, vocab_file, sorted_vocab=1)

If you want to load the ensemble embedding, you similarly need to provide the context embedding file and set ensemble to 1 in `load_wordrank_model` method.

In [ ]:
wr_context_file = 'wordrank.contexts'
model = Wordrank.load_wordrank_model(wr_word_embedding, vocab_file, wr_context_file, sorted_vocab=1, ensemble=1)

You can save these sorted embeddings using the standard gensim methods.

In [ ]:
from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file
model.save(temp_path)  # save the model

# Evaluating models
Now that the embeddings are loaded in Word2Vec format and sorted according to the word frequencies in corpus, you can use the evaluations provided by gensim on this model.

For example, it can be evaluated on following Word Analogies and Word Similarity benchmarks. 

In [ ]:
word_analogies_file = 'datasets/questions-words.txt'
model.accuracy(word_analogies_file)

In [ ]:
word_similarity_file = 'datasets/ws-353.txt'
model.evaluate_word_pairs(word_similarity_file)

These methods take an [optional parameter](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.accuracy) restrict_vocab which limits which test examples are to be considered.

The results here don't look good because the training corpus is very small. To get meaningful results one needs to train on 500k+ words.

# Conclusion
We learned to use Wordrank wrapper on a sample corpus and also how to directly load the Wordrank embedding files in gensim. Once loaded, you can use the standard gensim methods on this embedding.